In [ ]:
## IMPORTS

import os
import sys
import re
import time
import math
from datetime import datetime
from io import StringIO


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from numpy.polynomial.polynomial import Polynomial
import scipy.integrate as integrate
import scipy.optimize as sco
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import openpyxl
import networkx as nx
import sklearn.metrics
from IPython.core.display import HTML
import plotly.io as pio
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl as opxl
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import time


sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\parallel-flow-distribution-pressure-loss\ansys')
sys.path.append(r'D:\seagu_OneDrive\Documents\GitHub\PVT-thermal-performance-model')
import jou_gen as jg
import ansys_py_bridge as apb
import ansys.fluent.core as pyfluent
import plot_bridge as pb
import model as ty
import proc as pr
import plot_functions_here as pfun
import heat_transfer as bht
import fluids as fds
import ht
import general as gen

In [ ]:
folder_path = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests'
hypotheses_path = folder_path + '\\' + 'Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'test-Conditions-AR.xlsx'
geometry_path = folder_path + '\\' + 'Inputs_TEST_V4.5_GG.xlsx'
plot_path = folder_path + '\\' + 'Plot_hyp.xlsx'

hyp = pr.create_dict_from_excel(hypotheses_path,'Main')
steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre
stepConditions = steadyStateConditions_df.to_dict(orient='records')
panelSpecs = pr.import_geometry(geometry_path)

plot_hyp = pr.create_dict_from_excel(plot_path,'Main')

In [ ]:
plot_hyp['method'] = 'case'
plot_hyp['no_case'] = 7
plot_hyp['no_mesh'] = 0
plot_hyp['nb_it']=7
plot_hyp

In [ ]:
ht_tot_mesh_case_list, ht_rad_mesh_case_list, ht_conv_mesh_case_list, CFD_mesh_case_list, df_one_mesh_case_list, slices_df_mesh_case_list, PyFluent_mesh_case_list = pb.get_data(plot_hyp, panelSpecs, hyp, stepConditions)

In [ ]:
ht_tot_mesh_case_list[6]

In [ ]:
no_mesh = plot_hyp['no_mesh']
no_case = plot_hyp['no_case']
tot_ratio = ht_tot_mesh_case_list[0][ht_tot_mesh_case_list[0]['Component'].isin(['pv_front']) ]['ht'].values[0]
source_ratio = ht_tot_mesh_case_list[0][ht_tot_mesh_case_list[0]['Component'].isin(['User Energy Source']) ]['ht'].values[0]
rad_ratio = ht_rad_mesh_case_list[0][ht_rad_mesh_case_list[0]['Component'].isin(['pv_front']) ]['rad_ht'].values[0]
conv_ratio = ht_conv_mesh_case_list[0][ht_conv_mesh_case_list[0]['Component'].isin(['pv_front']) ]['conv_ht'].values[0]
ratio_rad = rad_ratio/(tot_ratio-source_ratio)
ratio_conv = conv_ratio/(tot_ratio-source_ratio)
ratio_source = -source_ratio/(tot_ratio-source_ratio)
print(ratio_rad+ ratio_conv+ ratio_source)

In [ ]:
Qdot_tube_fluid, Qdot_top, Qdot_top_rad, Qdot_tube_back, Qdot_PV_sky = pb.calculate_Qdot(plot_hyp, panelSpecs, hyp, stepConditions, mesh = 0, case = 0, iteration = 0)

In [ ]:
pb.plot_CFD_last_it('Qdot_top_conv', plot_hyp, panelSpecs, hyp, stepConditions)

In [ ]:
pb.plot_1D_last_it('Qdot_PV_sky', plot_hyp, panelSpecs, hyp, stepConditions)

In [ ]:
plot_hyp['method'] = 'ref'
pb.plot_big_it('Qdot_tube_fluid', plot_hyp, panelSpecs, hyp, stepConditions) 
## PV_sky mal implémenté 
## Qdot_top_rad revoir comment le faire

In [ ]:
plot_hyp['method'] = 'case'
plot_hyp['no_case'] = 7
plot_hyp['no_mesh'] = 0
pb.plot_big_it('Qdot_top_rad', plot_hyp, panelSpecs, hyp, stepConditions) 
## PV_sky mal implémenté 
## Qdot_top_rad revoir comment le faire

In [ ]:
plot_hyp['method'] = 'ref'
ht_tot_AR_list, ht_rad_AR_list, ht_conv_AR_list, CFD_AR_list, df_one_AR_list, slices_df_AR_list, PyFluent_AR_list, ht_tot_uniform, ht_rad_uniform, ht_conv_uniform, CFD_uniform, df_one_uniform, slices_df_uniform, df_PyFluent_uniform, df_one_1D, slices_df_1D, df_PyFluent_1D = pb.get_data(plot_hyp, panelSpecs, hyp, stepConditions)


In [ ]:
pb.plot_profile_temp(plot_hyp, panelSpecs, hyp, stepConditions)

In [ ]:
plot_hyp

In [ ]:
plot_hyp['method'] = 'mesh'
plot_hyp['nb_cases'] = 5
plot_hyp['nb_mesh'] = 1
plot_hyp['nb_it'] = 7
plot_hyp['folder_mesh'] = 'Simu_M'
plot_hyp['folder_name'] = 'cas_'
plot_hyp['new_save'] = True
pb.plot_1D_DeltaT_tot('Qdot_tube_fluid', plot_hyp, panelSpecs, hyp, stepConditions)
# pb.plot_1D_DeltaT_part('Qdot_tube_fluid', plot_hyp, panelSpecs, hyp, stepConditions)
## Fonctionne pas trop 

In [ ]:
plot_hyp['method'] = 'mesh'
plot_hyp['nb_cases'] = 5
plot_hyp['nb_mesh'] = 1
plot_hyp['nb_it'] = 1
plot_hyp['folder_mesh'] = 'Simu_1D_'
plot_hyp['folder_name'] = 'cas_'
plot_hyp['new_save'] = False
pb.plot_1D_DeltaT_tot('Qdot_tube_fluid', plot_hyp, panelSpecs, hyp, stepConditions)
# ht_tot_mesh_case_list, ht_rad_mesh_case_list, ht_conv_mesh_case_list, CFD_mesh_case_list, df_one_mesh_case_list, slices_df_mesh_case_list, PyFluent_mesh_case_list = pb.get_data(plot_hyp, panelSpecs, hyp, stepConditions)
# df_one_mesh_case_list[0][0][0]

In [ ]:
plot_hyp['method'] = 'mesh'
plot_hyp['nb_cases'] = 3
plot_hyp['nb_mesh'] = 1
plot_hyp['nb_it'] = 1
plot_hyp['folder_mesh'] = 'Simu_uniform'
plot_hyp['folder_name'] = 'cas_M'
plot_hyp['new_save'] = False
pb.plot_CFD_last_it('Qdot_top_conv', plot_hyp, panelSpecs, hyp, stepConditions)

In [ ]:
file_name = 'residuals_cas_ref_CFD_uniform.txt'
file_path_base = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests\cas_M3'
file_path = os.path.join(file_path_base, file_name)
residuals = pb.extract_residuals(file_path)
pd.DataFrame(residuals[0]).plot()

In [ ]:
file_name = 'residuals_cas_ref_CFD_uniform.txt'
file_path_M0 = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests\Simu_uniform1\cas_M0'
file_path_M1 = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests\Simu_uniform1\cas_M1'
file_path_M2 = r'D:\ANSYS Fluent Projects\pvt_slice_outdoor\Fluent_GMI\tests\Simu_uniform1\cas_M2'
os.path.join(file_path_M0, file_name)
residualsM0 = pb.extract_residuals(os.path.join(file_path_M0, file_name))
residualsM1 = pb.extract_residuals(os.path.join(file_path_M1, file_name))
residualsM2 = pb.extract_residuals(os.path.join(file_path_M2, file_name))
residuals = []
for i in range(5): 
    residuals.append(pd.concat([residualsM0[i], residualsM1[i], residualsM2[i]], axis=1))
    pd.DataFrame(residuals[i]).plot()

In [ ]:
    fig = go.Figure()
    
    for column in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[column], mode='lines', name=column))
    
    fig.update_layout(title='Graph of Residuals',
                      xaxis_title='Iteration',
                      yaxis_title='Value',
                      yaxis_type='log',  # Set the y-axis to logarithmic scale
                      legend_title='Residuals',
                      template='plotly_dark')
    
    fig.show()